''' This program runs a dynamic topic model

#Date: Mar 4, 2019

#Author: Carly Knight

#https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/dtm_example.ipynb
#https://stackoverflow.com/questions/50413059/dynamic-topic-modeling-with-gensim-which-code'''


## set up

In [1]:
import logging
import os
from gensim import corpora, utils
from gensim.models.wrappers.dtmmodel import DtmModel
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
from collections import defaultdict



### dtm

In [2]:
# you can also copy the path down directly. Change this variable to your DTM executable before running.
dtm_path = "/usr/local/bin/dtm-darwin64"

# Open corpus

In [3]:
file_loc = "/Users/carlyknight/Documents/Data/Annual Report/report_paragraphs/future_texts/"

#limit to over 1930
metadata = pd.read_csv(file_loc + "metadata_futureperfect.csv") 
metadata = metadata[(metadata['Year'] >= 1930)]
metadata['text_filename'] = metadata['Filename'].str.replace(".xml", ".txt")

#pick relevant files
textfiles = [i for i in metadata['text_filename'].values.tolist()]

In [4]:
#open files
for index, row in metadata.iterrows():
    txtfile= row['Filename'].replace(".xml", ".txt")
    #read in text file and turn into new variable
    with open(file_loc + txtfile, 'r') as f:
        metadata.loc[index,'text']= f.read()
        f.close()

In [5]:
#metadata.shape (36479,17)

In [6]:
#remove puncutation
#metadata['text'] = metadata['text'].str.replace(r'[^\w\s]+', '')

#remove numbers
#metadata['text'] = metadata['text'].str.replace(r'[\d]+', '')

#remove extraspaces
#metadata['text'] = metadata['text'].str.replace(r'\s+', ' ')

#turn to text
documents = metadata['text'].tolist()

## Preprocess

In [7]:
from gensim.parsing.preprocessing import preprocess_string

#processed corpus: removes punctuation, whitespaces, stopwords, stems, numbes
processed_corpus = [preprocess_string(document) for document in documents] 

#remove low frequency terms (terms with fewer than 20) (brings it down to 35,244 from 953,010 tokens)
frequency = defaultdict(int)

for text in processed_corpus:
    for token in text:
        frequency[token] += 1

processed_corpus = [
    [token for token in text if frequency[token] > 10]
    for text in processed_corpus
]

#dictionary
dictionary = Dictionary(processed_corpus)

#filter extremes: get rid of words that appear in no fewer than 5 documents
#dictionary.filter_extremes(no_below=5)
#len(dictionary) #46,729

#Bag of Words
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]

#Time slice
counts = metadata[['Year', "Filename"]].groupby(['Year']).agg(['count'])
timeslice=[i[0] for i in counts.values.tolist()]

# DTM

In [ ]:
model = DtmModel(dtm_path, bow_corpus, timeslice, num_topics=15,
                 id2word=dictionary, initialize_lda=True)

In [9]:
topics = model.show_topic(topicid=1, time=10, topn=30)
topics

[(0.022748591414139326, 'compani'),
 (0.010523767394516399, 'stock'),
 (0.008981057365953943, 'plan'),
 (0.007928191906834229, 'year'),
 (0.007522510467799447, 'share'),
 (0.006625967050506597, 'corpor'),
 (0.006356611436561772, 'tax'),
 (0.0062845500284043525, 'date'),
 (0.005825819383503139, 'agreement'),
 (0.005668637721945956, 'shall'),
 (0.005638820542117108, 'time'),
 (0.005387556481017257, 'payment'),
 (0.0049422367973985205, 'provid'),
 (0.0047462428414317605, 'includ'),
 (0.004661452364670406, 'option'),
 (0.004655775078875385, 'oper'),
 (0.00464098721546299, 'decemb'),
 (0.004431391099738042, 'director'),
 (0.004397335956119495, 'requir'),
 (0.004354318548929526, 'servic'),
 (0.004320850150401701, 'effect'),
 (0.004259768991550486, 'secur'),
 (0.004248792906599707, 'subsidiari'),
 (0.0041214638081043704, 'addit'),
 (0.004092770380102696, 'common'),
 (0.004079296065370617, 'period'),
 (0.004024404656092644, 'incom'),
 (0.00395142403005248, 'execut'),
 (0.003800545982640399, 'b

In [19]:
#https://towardsdatascience.com/topic-modeling-with-gensim-a5609cefccc

# save

In [10]:
import pickle

In [11]:
f = open('/Users/carlyknight/Documents/Data/FuturePerfect/dynamic_topic_models/topic15.pkl', 'wb') 
pickle.dump(model, f , protocol=4)          
f.close()                 